<a href="https://colab.research.google.com/github/ricktorzynski/tensorflow-drupal/blob/main/Book_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

import tensorflow.keras as tf


In [ ]:
ratings_df = pd.read_csv("book-data/ratings.csv")
books_df = pd.read_csv("book-data/books.csv")

In [ ]:
model = tf.models.load_model('model')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        300030      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 30)        1602750     input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
# Extract embeddings
book_em = model.get_layer('embedding')
book_em_weights = book_em.get_weights()[0]
book_em_weights.shape


(10001, 30)

In [ ]:
books_df_copy = books_df.copy()
books_df_copy = books_df_copy.set_index("book_id")

In [ ]:
b_id =list(ratings_df.book_id.unique())
b_id.remove(10000)
dict_map = {}
for i in b_id:
    dict_map[i] = books_df_copy.iloc[i]['title']
    
out_v = open('vecs.tsv', 'w')
out_m = open('meta.tsv', 'w')
for i in b_id:
    book = dict_map[i]
    embeddings = book_em_weights[i]
    out_m.write(book + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    
out_v.close()
out_m.close()

In [ ]:
#Making recommendations for user 100
book_arr = np.array(b_id) #get all book IDs
user = np.array([100 for i in range(len(b_id))])
pred = model.predict([book_arr, user])
pred

array([[3.4477377],
       [3.1564796],
       [3.305841 ],
       ...,
       [3.107275 ],
       [3.1836162],
       [3.0474133]], dtype=float32)

In [ ]:
pred = pred.reshape(-1) #reshape to single dimension
pred_ids = (-pred).argsort()[0:5]
pred_ids

array([ 479,  950, 7886, 4388,  948])

In [ ]:
books_df.iloc[pred_ids]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
479,480,18122,18122,1774510,42,440238153,9.780440e+12,Philip Pullman,2000.0,The Amber Spyglass,"The Amber Spyglass (His Dark Materials, #3)",eng,4.06,192315,217610,6693,4739,11883,39131,70648,91209,https://images.gr-assets.com/books/1329189152m...,https://images.gr-assets.com/books/1329189152s...
950,951,22283,7076703,1602338,63,009928264X,9.780099e+12,Chuck Palahniuk,1999.0,Survivor,Survivor,en-US,3.93,49551,88231,3078,903,4759,20844,34767,26958,https://images.gr-assets.com/books/1327346693m...,https://images.gr-assets.com/books/1327346693s...
7886,7887,13451,13451,3048970,28,067103264X,9.780671e+12,Stephen King,1999.0,Storm of the Century: An Original Screenplay,Storm of the Century: An Original Screenplay,NaN,3.86,14629,15343,265,257,1120,4087,5005,4874,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
4388,4389,2815949,2815949,2841882,6,1400054354,9.781400e+12,"Ina Garten, Quentin Bacon",2008.0,Barefoot Contessa Back to Basics,Barefoot Contessa Back to Basics,eng,4.23,22228,22305,199,654,708,2960,6541,11442,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
948,949,13895,13895,588034,56,1857232097,9.781857e+12,Robert Jordan,1993.0,The Fires of Heaven,"The Fires of Heaven (Wheel of Time, #5)",en-US,4.13,93227,104317,1864,863,3367,18885,39542,41660,https://images.gr-assets.com/books/1327866286m...,https://images.gr-assets.com/books/1327866286s...


In [ ]:
web_book_data = books_df[["book_id", "title", "image_url", "authors", "original_publication_year"]]
web_book_data = web_book_data.sort_values('book_id')
web_book_data.head()

,book_id,title,image_url,authors,original_publication_year
0,1,"The Hunger Games (The Hunger Games, #1)",https://images.gr-assets.com/books/1447303603m...,Suzanne Collins,2008.0
1,2,Harry Potter and the Sorcerer's Stone (Harry P...,https://images.gr-assets.com/books/1474154022m...,"J.K. Rowling, Mary GrandPré",1997.0
2,3,"Twilight (Twilight, #1)",https://images.gr-assets.com/books/1361039443m...,Stephenie Meyer,2005.0
3,4,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680m...,Harper Lee,1960.0
4,5,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...,F. Scott Fitzgerald,1925.0


In [ ]:
web_book_data.to_json(r'web_book_data.json', orient='records')